In [1]:
import pandas as pd
from transformers import pipeline
classification_task = pipeline("sentiment-analysis",model = "distilbert-base-uncased-finetuned-sst-2-english")
unmasker = pipeline('fill-mask', model='bert-base-uncased')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string 
result = string.punctuation 
from sentence_transformers import SentenceTransformer, util
import numpy as np
model = SentenceTransformer('stsb-roberta-large')
from tabulate import tabulate
import ipysheet
import csv
from timeit import default_timer as timer
import time
import homoglyphs as hg
from itertools import combinations
pyTorchData = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
bertAttackData = pd.read_csv('https://raw.githubusercontent.com/LinyangLee/BERT-Attack/master/data_defense/imdb_1k.tsv', delimiter='\t', header=None)
batch1=pyTorchData[0:5000]
sentences = batch1[0]#here we can easily add the adversarial sentences

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jonathan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#for regular
allLabels = {}
dictClassifications = {}
totalTime = 0
for i in range(len(sentences)):
    start = timer()
    a = classification_task(sentences[i])
    end=timer()
    totalTime+=(end-start)
    dictClassifications[i]=a,sentences[i],batch1[1][i],(end-start)
for i in dictClassifications:
    correct = False
    if dictClassifications[i][0][0]["label"] == "POSITIVE":
        predictedLabel = 1
    else:
        predictedLabel = 0
    givenLabel = dictClassifications[i][2]
    if predictedLabel==givenLabel:
        correct = True
    allLabels[i]=sentences[i],dictClassifications[i][0],predictedLabel,givenLabel,correct,dictClassifications[i][3]
totalWrong = 0
wrongSentences = []
for i in allLabels:
    correct = allLabels[i][-1]
    if not correct:
        wrongLabel = allLabels[i][1][0]["label"]
        wrongConfidence = allLabels[i][1][0]["score"]
        wrongSentences.append((sentences[i],wrongLabel,wrongConfidence,i))
        
        totalWrong+=1
percentageRight = (len(allLabels)-totalWrong)/len(allLabels)

In [3]:
col_names = ["Sentence", "Actual Label","Predicted Label","Confidence","Accuracy","Runtime"]
data = []
for i in range(len(allLabels)):
    sentence = allLabels[i][0]
    confidence = allLabels[i][1][0]['score']
    runtime = allLabels[i][5]
    if allLabels[i][2]==1:
        predicted = 'Positive'
    elif allLabels[i][2]==0:
        predicted = 'Negative'
    if allLabels[i][3]==1:
        actual = 'Positive'
    elif allLabels[i][3]==0:
        actual = 'Negative'
    if allLabels[i][4]:
        correct = "Correct"
    else:
        correct = "False"
    data.append([sentence,actual,predicted,confidence,correct,runtime])
    


In [4]:
truePositives = []
trueNegatives = []
falsePositives = []
falseNegatives = []
for i in range(len(data)):
    if data[i][4]=="False":
        if data[i][2]=='Positive':
            falsePositives.append(i)
        else:
            falseNegatives.append(i)
    else:
        if data[i][2]=='Positive':
            truePositives.append(i)
        else:
            trueNegatives.append(i)

In [5]:
numTruePositives = len(truePositives)
numTrueNegatives = len(trueNegatives)
numFalsePositives = len(falsePositives)
numFalseNegatives = len(falseNegatives)
accuracy = (numTruePositives+numTrueNegatives)/(numTruePositives+numTrueNegatives+numFalsePositives+numFalseNegatives)
precision = numTruePositives/(numTruePositives+numFalsePositives)
recall = numTruePositives/(numTruePositives+numFalseNegatives)
specificity =numTrueNegatives/(numTrueNegatives+numFalsePositives)
f1 = 2*(precision*recall)/(precision+recall)

In [6]:
filename = "Original_PyTorch_VPN.csv"#NEED TO CHANGE IF DIFF DATASET
col_names = ["Sentence", "Actual Label","Predicted Label","Confidence","Accuracy","Changed Word","Runtime"]
row1 = [f"Accuracy:{accuracy:.3f}",f"Precision:{precision:.3f}",f"Recall:{recall:.3f}",f"Specificity:{specificity:.3f}",f"F1-Score:{f1:.3f}",f"Total Runtime:{totalTime:.3f}"]
with open(filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
     
    # writing the fields
    csvwriter.writerow(row1)
    csvwriter.writerow(col_names)
     
    # writing the data rows
    csvwriter.writerows(data)